<a href="https://colab.research.google.com/github/Diyon335/PRA3024/blob/master/Week3_Gideon/Final_week3_fisherMatrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 1 and 2
This section computes the standard deviation for any observables. The code is generalised as to allow the user to change their Power Spectral Density functions and Waveforms. 

In [0]:
#Import the necessary libraries 
import numpy as np
import cmath as cm
import sympy as sp

In [0]:
#Created an integration and differentiation function
def integral_of(function, low, high, dx):
  area = 0
  x = low
  steps = int((high-low)/dx)
  for i in range(steps):
    area += function(x)*dx
    x+=dx
  return area


In [0]:
#Declare any constants
## PSD ##
f0 = 150
fs = 40 
## Wavefunction h(f) ##
solar_mass = 2*(10**30)
m1 = 2*solar_mass
m2 = 1*solar_mass
M = m1 + m2
SNR = 10
eta = (m1*m2)/(M**2)
tc = 1
phic = 1

#If you want higher PN orders, then add more corrections. But dont forget to add it into the pn_array below
a0 = 1
a1 = 0
a2 = (20/9)*(743/336 + (11/4)*eta)

#The variables with which h(f) will be differentiated wrt
variables = [tc,phic,eta,M]
#change to match with variables
observables= ["tc","phic","eta","M"]

In [0]:
#If any more corrections are added above, then add them to this array
pn_array = [a0,a1,a2]

In [0]:
#This is the function for the phase, xi(f)
def xi(f):
  summed_function = 0
  v = (((float(sp.pi))*M*f)**(1/3))
  for j in range(0, len(pn_array)):
    summed_function += pn_array[j]*(v**(j))
  return 2*float(sp.pi)*f*tc - phic - float(sp.pi)/4 + (3/(128*eta*(v**5)))*summed_function

In [0]:
#Here you can initialise any PSD function
#This is for the Initial LIGO / the fs and f0 are initialised above
def s_f(f):
  S0 = 9*10**(-46)
  x = f/f0
  return S0*((4.49*x)**(-5) + (0.16*x)**(-4.52) + 0.52 + (0.32*x)**2 )

In [0]:
#The waveform and its conjugate
def h_real(f):
  A = sp.sqrt(  SNR/ (4* integral_of(s_f,fs,f0,0.01) ) )
  xi_f = xi(f)
  return A*f**(-7/6)*sp.exp(1j*xi_f)

def h_compl(f):
  A = sp.sqrt(  SNR/ (4* integral_of(s_f,fs,f0,0.01) ) )
  xi_f = xi(f)
  return A*f**(-7/6)*sp.exp(-1j*xi_f)

In [0]:
#This returns the derivative of the real and complex waveforms respectively 
def d_h_real(i,inc):
  return (h_real(variables[i]+inc) - h_real(variables[i])) / inc
def d_h_compl(j,inc):
  return (h_compl(variables[j]+inc) - h_compl(variables[j])) / inc

Unfortunately, due to errors when computing any values OTHER than the diagonals, only the diagonal elements were computed. Weather this affected any results is unknown due to the lack of understanding of theory. Sorry if that is the case!

In [0]:
def create_fm(matrix, increment):
  #This part here creates the range of frequencies to integrate from
  frequencies = np.zeros(shape=(len(variables)*len(variables)))
  start_freq = fs
  f_steps = int(  (f0-fs)  /  (len(variables)*len(variables))    )
  for s in range(len(variables)*len(variables)):
    frequencies[s] = start_freq
    start_freq += f_steps
  
  f_index = 0
  intg = 0
  for i in range(len(variables)):
    for j in range(len(variables)):
      if (i==j):
        intg += float(4*d_h_real(i,increment)*d_h_compl(j,increment)/s_f(frequencies[f_index]))*increment
        matrix[i][j] = intg
      f_index += 1
  
  return matrix

In [15]:
#Create a matrix for the values to go into
fm = np.zeros( shape=(len(variables),len(variables)) )

create_fm(fm, 0.01)
print(fm)

[[1.82694170e+77 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.47490703e+78 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 5.98154982e+80 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 5.98154982e+80]]


In [0]:
#This function will compute the standard deviation of measuring observables (tc, phic, eta, M)
def compute_stds(matrix):
  inverse = np.linalg.inv(matrix)
  diag_elements = []
  for i in range(len(variables)):
    for j in range(len(variables)):
      if (i==j):
        diag_elements.append(inverse[i][j])
  for k in range(len(diag_elements)):
    diag_elements[k] = np.sqrt(diag_elements[k])

  for l in range(len(diag_elements)):
    print("The standard deviation for {} is {}".format(observables[l],diag_elements[l]))



Note: The following values of standard deviation are computed without accounting for the value of c and G. The following values set c = G = 1

In [17]:
compute_stds(fm)

The standard deviation for tc is 2.3395786802596234e-39
The standard deviation for phic is 6.3565366507702305e-40
The standard deviation for eta is 4.0887742972236405e-41
The standard deviation for M is 4.0887742972236405e-41


## Exercise 3
What if we tried with M_chirp instead?

In [0]:
M_chirp = 